# 검색 성능 벤치마크 (Recall, NDCG, Latency)

검색 시스템의 성능을 정량적으로 평가하기 위한 핵심 메트릭을 구현합니다.

## 학습 목표

1. **Recall@k**: 재현율 - 관련 문서를 얼마나 찾아내는가
2. **NDCG@k**: 정규화 순위 기반 품질 - 상위 순위에 관련 문서가 있는가
3. **Latency**: 응답 시간 - 얼마나 빠른가
4. **실전 평가**: Ground Truth 기반 정량 평가

## 메트릭 개념

### Recall@k (재현율)
```
Recall@k = (검색된 관련 문서 수) / (전체 관련 문서 수)
```
- 값 범위: 0.0 ~ 1.0
- 높을수록 좋음 (1.0 = 모든 관련 문서 검색)
- 예: 10개 관련 문서 중 8개 검색 → Recall@10 = 0.8

### NDCG@k (Normalized Discounted Cumulative Gain)
```
DCG@k = Σ (rel_i / log2(i + 1))
NDCG@k = DCG@k / IDCG@k
```
- 값 범위: 0.0 ~ 1.0
- 순위를 고려한 품질 측정
- 상위에 관련 문서가 많을수록 높음

### Latency (레이턴시)
```
Latency = 검색 종료 시간 - 검색 시작 시간
```
- 단위: 밀리초 (ms)
- 낮을수록 좋음
- p50, p95, p99 백분위수로 측정

## 참고 자료

- [Information Retrieval Metrics](https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval))
- [NDCG 설명](https://en.wikipedia.org/wiki/Discounted_cumulative_gain)


In [3]:
# ============================================================================
# 1. 벤치마크 메트릭 구현
# ============================================================================
import time

import numpy as np
from langchain_core.retrievers import BaseRetriever


def calculate_recall_at_k(
    retrieved_ids: list[int],
    relevant_ids: list[int],
    k: int,
) -> float:
    """
    Recall@k 계산

    Args:
        retrieved_ids: 검색된 문서 ID 리스트
        relevant_ids: 관련 있는 문서 ID 리스트 (Ground Truth)
        k: Top-k

    Returns:
        Recall@k (0.0 ~ 1.0)
    """
    if not relevant_ids:
        return 0.0

    # Top-k만 고려
    retrieved_k = retrieved_ids[:k]

    # 관련 문서 중 검색된 개수
    retrieved_relevant = set(retrieved_k) & set(relevant_ids)

    recall = len(retrieved_relevant) / len(relevant_ids)

    return recall


def calculate_dcg_at_k(
    retrieved_ids: list[int],
    relevance_scores: dict,
    k: int,
) -> float:
    """
    DCG@k (Discounted Cumulative Gain) 계산

    Args:
        retrieved_ids: 검색된 문서 ID 리스트
        relevance_scores: {doc_id: relevance} (0=무관, 1=관련, 2=매우관련)
        k: Top-k

    Returns:
        DCG@k
    """
    dcg = 0.0

    for i, doc_id in enumerate(retrieved_ids[:k], start=1):
        relevance = relevance_scores.get(doc_id, 0)
        # DCG 공식: rel_i / log2(i + 1)
        dcg += relevance / np.log2(i + 1)

    return dcg


def calculate_ndcg_at_k(
    retrieved_ids: list[int],
    relevance_scores: dict,
    k: int,
) -> float:
    """
    NDCG@k (Normalized DCG) 계산

    Args:
        retrieved_ids: 검색된 문서 ID 리스트
        relevance_scores: {doc_id: relevance}
        k: Top-k

    Returns:
        NDCG@k (0.0 ~ 1.0)
    """
    # 실제 DCG
    dcg = calculate_dcg_at_k(retrieved_ids, relevance_scores, k)

    # 이상적인 DCG (Ideal DCG)
    # 관련도 높은 순으로 정렬
    sorted_relevance = sorted(
        relevance_scores.values(),
        reverse=True,
    )

    ideal_dcg = 0.0
    for i, rel in enumerate(sorted_relevance[:k], start=1):
        ideal_dcg += rel / np.log2(i + 1)

    # 정규화
    if ideal_dcg == 0:
        return 0.0

    ndcg = dcg / ideal_dcg

    return ndcg


def measure_latency(
    retriever: BaseRetriever,
    queries: list[str],
    warmup: int = 2,
) -> dict:
    """
    레이턴시 측정

    Args:
        retriever: 검색기
        queries: 쿼리 리스트
        warmup: 워밍업 쿼리 개수

    Returns:
        {"mean": float, "p50": float, "p95": float, "p99": float}
    """
    latencies = []

    # 워밍업
    for query in queries[:warmup]:
        _ = retriever.invoke(query)

    # 실제 측정
    for query in queries[warmup:]:
        start_time = time.time()
        _ = retriever.invoke(query)
        end_time = time.time()

        latency_ms = (end_time - start_time) * 1000
        latencies.append(latency_ms)

    if not latencies:
        return {"mean": 0.0, "p50": 0.0, "p95": 0.0, "p99": 0.0}

    return {
        "mean": np.mean(latencies),
        "p50": np.percentile(latencies, 50),
        "p95": np.percentile(latencies, 95),
        "p99": np.percentile(latencies, 99),
    }
